# Интеграция планирования и обучения с подкреплением
## OpenTalks.AI 2021
### Кирилл Аксёнов

## Часть 1: Отличия планирования и RL

### RL:
В RL зачастую цнль состоит в том, чтобы максимизировать награду (дисконтированное будущее накопленное вознаграждение, или, проще говоря, вознаграждение в долгосрочной перспективе).

Другой ключевой особенностью обучения с подкреплением является то, что оно явно рассматривает всю проблему целенаправленного взаимодействия агента с неопределенной средой.

В RL есть явное понятие агента, среды и цели, и награда - единственный сигнал, который сообщает агенту, насколько хорошо он работает, но награда не сообщает агенту, какие действия он должен предпринять в каждом конкретном случае.

<img src="https://github.com/hse-ds/iad-applied-ds/raw/master/2020/seminars/seminar15/rlIntro.png" caption="Взаимодействия агента со средой" style="width: 500px;" />



### Планирование
Планирование же часто выполняется «офлайн», то есть перед выполнением. Пока выполняется план, он зачастую остается неизменным. Однако, во многих случаях это нежелательно, поскольку может потребоваться реакция на изменение среды. Ввиду планирования поведения до конца эпизода, агенту гораздо проще достигать долгосрочных наград, особенно когда они разреженны.

### RL + Планирование:
RL и планирование довольно взаимосвязаны ввиду аналогичности проблем. Однако, в RL модель перехода и функция вознаграждения MDP  обычно неизвестны. Следовательно, единственный способ найти или оценить оптимальную стратегию, которая позволит агенту (почти оптимально) вести себя в этой среде, - это взаимодействовать со средой и собирать некоторую информацию о ее «динамике», после того как появиться модель среды возможно планирование. При таком подходе агент изучает среду в которой находится, для того чтобы в дальнейшем использовать полученные знания для планирования своего поведения с целью достижения максимальной награды.

## Часть 2: Среды и оболочки над ними

Важная часть в обучении с подкреплением это среда!

[Gym](https://gym.openai.com) $-$ это набор инструментов для разработки и сравнения алгоритмов обучения с подкреплением, который также включает в себя большой [набор окружений](https://gym.openai.com/envs/).

### Создание окружения

Для создания окружения используется функция ```gym.make(<имя окружения>)```. 

In [ ]:
import gym

# Создаем окружение
env = gym.make("Taxi-v3")

# Инициализируем окружение
state = env.reset()
print(f"state: {state}")

# Выполняем действие в среде 
next_state, r, done, info = env.step(0)
print(f"next_state: {next_state} , r: {r}, done: {done}, info: {info}")

# Закрываем окружение
env.close()

### Основные методы окружения:

* ``reset()`` $-$ инициализация окружения, возвращает первое наблюдение (состояние).  
* ``step(a)`` $-$ выполнить в среде действие $\mathbf{a}$ и получить кортеж: $\mathbf{\langle s_{t+1}, r_t, done, info \rangle}$, где $\mathbf{s_{t+1}}$ - следующее состояние, $\mathbf{r_t}$ - вознаграждение, $\mathbf{done}$ - флаг заверешния, $\mathbf{info}$ - дополнительная информация

### Дополнительные методы:
* ``render()`` $-$ визуализация текущего состояния среды (удобно, если мы запускаем локально, в колабе ничего не увидим)

* ``close()`` $-$ закрывает окружение 

### Свойства среды:
* ``env.observation_space`` $-$ информация о пространстве состояний
* ``env.action_space`` $-$ информация о пространстве действий


In [ ]:
print(f"env.observation_space: {env.observation_space}")
print(f"env.action_space: {env.action_space}")

### Среда ``Taxi-v3``

Информацию о любой среде можно найти в [исходниках](https://github.com/openai/gym/blob/master/gym/envs/classic_control/mountain_car.py) или на [сайте](https://gym.openai.com/envs/MountainCar-v0/). О ``Taxi-v3`` мы можем узнать следующее: 


#### Задание:
Есть 4 места (помечены разными буквами), задача состоит в том, чтобы забрать пассажира в одном месте и высадить его в другом. +20 баллов дается за успешный перевозку пассажира, но за каждый шаг теряется 1 балл. Также существует штраф в размере 10 баллов за неправильную посадку и высадку.

#### Пространство состояний  Discrete(500):

Имеется 500 дискретных состояний, поскольку имеется 25 мест, где может оказаться такси, 5 возможных местоположений пассажира (включая случай, когда пассажир находится в такси) и 4 места назначения.


#### Пространство действий Discrete(6):

Num | Action|
----|-------------|
0   | move north         |
1   | move east          |
2   | move west          |
3   | push left          |
4   | pickup passenger   |
5   | drop off passenger |

* Вознаграждения: 
     По умолчанию вознаграждение за шаг равно -1,
     кроме доставки пассажира(+20),
     или незаконное выполнение действий "посадка" и "высадка"(-10).

### Пример со случайной стратегией:

Для выбора действия используется ``env.action_space.sample()``

In [ ]:
import time
from IPython.display import clear_output

# создаем окружение
env = gym.make("Taxi-v3")
# добавляем wrapper (обертку), чтобы задать ограничение на число шагов в среде
env = gym.wrappers.TimeLimit(env, max_episode_steps=250)

# проводим инициализацию и запоминаем начальное состояние
s = env.reset()
env.render()
done = False

while not done:
    # выполняем действие, получаем s, r, done, info
    s, r, done, _ = env.step(env.action_space.sample())
    clear_output(wait=True)
    time.sleep(0.1)
    env.render()
    
env.close()

# Сначала закрываем окружение, чтобы видео записалось полностью
env.close()

### Оболочки
#### Reward Wrapper
Мы знаем, что окружения создаются с помощью команды ``gym.make(<имя среды>)``, но что если мы хотим немного изменить окружение или добавить какой-то дополнительный функционал? Для этого существуют обертки (wrappers). Рассмотрим среду ``Taxi-v3``, предположим, что мы хотим поменять вознаграждения на следующие: 1 за решение задачи, -1 за неправильную посадку/высадку пассажира и 0 во всех остальных случаях. Для этого мы можем воспользоваться ``RewardWrapper``-ом:

In [ ]:
class MyRewardWrapper(gym.RewardWrapper):

    def reward(self, reward):
        if reward == -1:
            return 0
        elif reward == 20:
            return 1
        elif reward == -10:
            return -1
        else: 
            raise KeyError

In [ ]:
env = MyRewardWrapper(gym.make("Taxi-v3"))
observation = env.reset()

rewards = set()

while True:
    observation, reward, done, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if done:
        break

# выведем все вознаграждения, которые получал агент
print(rewards)
env.close()

#### Time Limit Wrapper

В зависимости от рандома, мы могли получить разные результаты, но обычно это ``{0, -1}``. Откуда такой результат? Ведь среда заканчивается только когда задание выполнено. Все дело во встроенной обертке ограничивающей максимальное количество шагов. 

In [ ]:
env = gym.make("Taxi-v3")
print(type(env))
env.close()

Можно воспользоваться окружением без этой обертки, вызвав ``.env``:

In [ ]:
env = gym.make("Taxi-v3").env
env = MyRewardWrapper(env)

observation = env.reset()
rewards = set()

while True:
    observation, reward, done, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if done:
        break

print(rewards)
env.close()

А если у нас есть окружение без этого враппера по умолчанию, то можно добавить его вот так:

In [ ]:
env = gym.make("Taxi-v3").env
env = MyRewardWrapper(env)
env = gym.wrappers.TimeLimit(env, max_episode_steps=1)

observation = env.reset()
rewards = set()

while True:
    observation, reward, done, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if done:
        break

print(rewards)
env.close()

Чтобы различать два случая завершения среды, в ``info`` передается дополнительный ключ ``TimeLimit.truncated: True``, если среду завершил ``TimeLimit`` враппер:

In [ ]:
env = gym.make("Taxi-v3")
env = gym.wrappers.TimeLimit(env, max_episode_steps=1)

env.reset()
_, _, _, info = env.step(env.action_space.sample())

print(info)
env.close()

### Action Wrapper
Представим, что наш водитель находится не в лучшем своем состоянии и независимо от выбора агента, в 50% случаев совершает случайные действий. Сделать среду стохастическои и добиться такого эффекта мы можем, используя ``ActionWrapper``:

In [ ]:
import numpy as np

class TaxiRandomActionWrapper(gym.ActionWrapper):

    def __init__(self, env, probability=0.5):
        super().__init__(env) 
        self.probability = probability
        

    def action(self, action):
        if np.random.random() < self.probability:
            return env.action_space.sample()
        else: 
            return action

Чтобы проверить, что обертка работает будем выполнять единственное действие:

In [ ]:
env = gym.make("Taxi-v3").env
env = MyRewardWrapper(env)
env = TaxiRandomActionWrapper(env)

observation = env.reset()
rewards = set()

while True:
    action = 0
    observation, reward, done, info = env.step(action)
    rewards.add(reward)
    if done:
        break

print(rewards)
env.close()

### Wrapper

Класс ``gym.Wrapper`` является базовым для всех оберток. Подкласс может переопределить    многие методы для изменения поведения исходной среды, не изменяя при этом ее исходный код. Например, мы можем изменить метод step и добавить, какую-то дополнительную информацию в ``info``:

In [ ]:
class MyWrapper(gym.Wrapper):
    
    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        info['Wrapped'] = True
        return observation, reward, done, info

In [ ]:
env = gym.make("Taxi-v3")
env = MyWrapper(env)

env.reset()
_, _, _, info = env.step(env.action_space.sample())

print(info)
env.close()

## Часть 3: Dyna-Q

Давайте реализуем простейший алгоритм планирования, а именно табличный алгоритм Dyna-Q.

<img src="https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/10/dyna.png" style="width: 700px;">

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output

%matplotlib inline

In [ ]:
def show_progress(rewards_batch, log, reward_range=None):
    """
    Удобная функция, которая отображает прогресс обучения.
    """

    if reward_range is None:
        reward_range = [-990, +10]
    mean_reward = np.mean(rewards_batch)
    log.append([mean_reward])

    clear_output(True)
    plt.figure(figsize=[8, 4])
    plt.subplot(1, 2, 1)
    plt.plot(list(zip(*log))[0], label='Mean rewards')
    plt.legend(loc=4)
    plt.grid()
    plt.grid()
    plt.show()

In [ ]:
env = gym.make("Taxi-v3")
env.render()

# гиперпараметры алгоритма
alpha = 0.1
gamma = 0.95
epsilon = 0.1
episodes_number = 1001
on_model_updates = 10

In [ ]:
class Model:
    def __init__(self, env_):
        self.mask_state = np.zeros([env_.observation_space.n], dtype="int32")
        self.mask_state_action = np.zeros([env_.observation_space.n, env_.action_space.n], dtype="int32")
        self.r = np.zeros([env_.observation_space.n, env_.action_space.n])
        self.next_s = np.zeros([env_.observation_space.n, env_.action_space.n], dtype="int32")

    def add(self, s: int, a: int, r: float, next_s: int):
        self.mask_state[s] = 1
        self.mask_state_action[s][a] = 1
        self.r[s][a] = r 
        self.next_s[s][a] = next_s

    def sample(self) -> [int, int, float, int]:
        """
        returns s, a, r, next_s
        """
        s = np.random.choice(np.where(self.mask_state == 1)[0])
        a = np.random.choice(np.where(self.mask_state_action[s] == 1)[0])
        return s, a, self.r[s][a], self.next_s[s][a]

In [ ]:
def initialize_q_table(env):
    q_table_ = np.zeros([env.observation_space.n, env.action_space.n])
    return q_table_

# определяем память, в которой будет храниться Q(s,a)
q_table = initialize_q_table(env)
log = []
rewards_batch = []

# инициализируем модель
m = Model(env)

In [ ]:
def compute_q(alpha_, reward_, gamma_, next_max_, old_value_):
    return (1 - alpha_) * old_value_ + alpha_ * (reward_ + gamma_ * next_max_)


for i in range(1, episodes_number):
    state = env.reset()

    episode, reward, episode_reward = 0, 0, 0
    done = False

    while not done:
        # выбираем действие, используя eps-greedy исследование среды
        # с вероятностью epsilon выбираем случайное действие, иначе
        # выполняем действие жадно, согласно текущей Q-таблице
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # исследуем среду
        else:
            action = np.argmax(q_table[state])  # используем Q-функцию

        # выполняем действие в среде
        next_state, reward, done, info = env.step(action)
        m.add(state, action, reward, next_state)
        # получаем old_value (Q(s,a)) и next_max (max(Q(s', a')))
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        q_table[state, action] = compute_q(alpha_=alpha, reward_=reward, gamma_=gamma, next_max_=next_max,
                                           old_value_=old_value)
        state = next_state

        for _ in range(on_model_updates):
            m_s, m_a, m_r, m_next_s = m.sample()
            old_value = q_table[m_s, m_a]
            next_max = np.max(q_table[m_next_s])
            q_table[m_s, m_a] = compute_q(alpha_=alpha, reward_=m_r, gamma_=gamma, next_max_=next_max,
                                          old_value_=old_value)

        episode += 1
        episode_reward += reward
    rewards_batch.append(episode_reward)

    if i % 100 == 0:
        show_progress(rewards_batch, log)
        rewards_batch = []
        print(f"Episode: {i}, Reward: {episode_reward}")

## Часть 4: Monte-Carlo Tree Search(MCTS)

### Среда
#### О игре четыре в ряд
* Четыре в ряд - это пошаговая игра, в которой два игрока поочередно бросают цветные диски в вертикальную сетку. Цель каждого игрока - сформировать **последовательность из четырех дисков** в ряд первым. Это **игра с полной информацией**, то есть каждый игрок хорошо осведомлен обо всех событиях, которые произошли ранее.

<img src="https://upload.wikimedia.org/wikipedia/commons/a/ad/Connect_Four.gif">


* Четыре в ряд также можно рассматривать как **игру с нулевой суммой**.

### Среда Kaggle для загрузки ConnectX

* Kaggle предоставляет удивительно простую в использовании OpenAI-подобную среду для игры в четыре в ряд.

* Для окончательного тестирования агента также возможно использование предопределенных **random** и **negamax** агентов.

https://www.kaggle.com/ajeffries/connectx-getting-started

In [ ]:
#!pip install 'kaggle-environments>=0.1.6'

In [ ]:
from kaggle_environments import evaluate, make, utils
import random
import math
import time
import inspect
import os
import numpy as np

### Загрузка среды

In [ ]:
env = make("connectx", debug=True)
env.reset()

env.run(["random", "random"])
env.render(mode="ipython", width=500, height=450)

### Различные подходы к решению проблемы
Для решения этой проблемы может быть много разных алгоритмов, например
* Minimax (negamax)
* Minimax с альфа-бета отсечением
* Q Learning
* PPO
* Monte-Carlo Tree Search
* etc

Рассмотрим алгоритм поиска по дереву Монте-Карло. Основным узким местом для игр с большим пространством действий (7 в случае connect4) является то, что он требует обширного поиска с учетом различных перестановок и комбинаций игрового поля. MCTS пытается эффективно решить эту проблему, как объясняется ниже, за счет сокращения пространства поиска и в то же время сохранения эффективности.

### Поиск по дереву Монте-Карло: теория
**Идея**: поиск по дереву методом Монте-Карло строит дерево поиска с n узлами, каждый из которых содержит информацию о количестве побед и посещений. Первоначально дерево начинается с одного корневого узла и выполняет итерации до тех пор, пока не закончилось отведенной под поиск время.

MCTS состоит из четырех основных этапов
<img src = "https://miro.medium.com/max/6000/1*i4qhdo44WCF1YL25gneWmw.png" style="width: 500px;">


* **Выбор** - на этом этапе агент запускается с корневого узла, выбирает узел, используя стратегию по дереву (классический вариант подразумевает эпсилон-жадную), применяет выбранные действия и продолжает работу, пока не будет достигнуто конечное состояние. В нашем случае будем использовать эпсилон жадную стратегию.
* **Расширение** - когда найден узел, который до этого не посещался, дерево расширяется, чтобы включить неисследованный узел.
* **Симуляция** - после добавления узла алгоритм доигрывает партию(игру) до конца, при этом используется быстрая стратегия (зачастую случайная).
* **Обратное распространение** - когда агент достигает конечного состояния игры, обновляются все пройденные узлы, а именно статистика посещений и побед для каждого узла.

Вышеупомянутые шаги повторяются для некоторых итераций.

* Наконец, в качестве следующего действия выбирается дочерний элемент корневого узла с наибольшим числом посещений, поскольку чем больше число посещений, тем выше значение ucb.

### Работа MCTS
* Каждая итерация начинается с корня.
* Следуя стратегии по дереву, достигаем листового узла.
* Добавляем узел "N".
* Доигрываем партию используя случайную стратегию.
* Обновляем статистику в каждом посещенном узле.

### MCTS Agent to play connectX 

In [ ]:
def connectx_agent(obs, config):
    
    def put_new_piece(grid, col, mark, config):
        next_state = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if not next_state[row][col]:
                break
        next_state[row][col] = mark
        return next_state

    def check_result(grid, piece, config):

        def look_for_window(window):
            return (window.count(piece) == 4 and window.count(0) == config.inarow - 4)

        def calculate_windows():
            is_success = False
            sequences = ['horizontal', 'vertical', 'p_diagonal', 'n_diagonal']

            for sequence_type in sequences:

                if sequence_type == 'horizontal':
                    for row in range(config.rows):
                        for col in range(config.columns-(config.inarow-1)):
                            window = list(grid[row, col:col+config.inarow])
                            if look_for_window(window):
                                return True

                elif sequence_type == 'vertical':
                    for row in range(config.rows-(config.inarow-1)):
                        for col in range(config.columns):
                            window = list(grid[row:row+config.inarow, col])
                            if look_for_window(window):
                                return True

                elif sequence_type == 'p_diagonal':
                    for row in range(config.rows-(config.inarow-1)):
                        for col in range(config.columns-(config.inarow-1)):
                            window = list(
                                grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                            if look_for_window(window):
                                return True

                elif sequence_type == 'n_diagonal':
                    for row in range(config.inarow-1, config.rows):
                        for col in range(config.columns-(config.inarow-1)):
                            window = list(
                                grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                            if look_for_window(window):
                                return True

            return is_success

        return calculate_windows()

    class MCTS():

        def __init__(self, obs, config):

            self.state = np.asarray(obs.board).reshape(
                config.rows, config.columns)
            self.config = config
            self.player = obs.mark
            self.final_action = None
            self.time_limit = 4
            self.root_node = (0,)
            self.tunable_constant = 1.0
            self.value_type = 'epsilon-greedy'
            self.epsilon = 0.1

            self.tree = {self.root_node: {'state': self.state, 'player': self.player,
                                          'child': [], 'parent': None, 'total_node_visits': 0,
                                          'total_node_wins': 0}}
            self.total_parent_node_visits = 0
            
        def get_value(self, node_no):
            if not self.total_parent_node_visits:
                return math.inf
            else:
                if self.value_type == 'greedy' or self.value_type == 'epsilon-greedy':
                    value_estimate = self.tree[node_no]['total_node_wins'] / \
                        (self.tree[node_no]['total_node_visits'] + 1)
                    return value_estimate
#                 elif self.value_type == 'ucb':


        def selection(self):
            '''
            Aim - To select the leaf node with the maximum value
            '''
            is_terminal_state = False
            leaf_node_id = (0,)
            while not is_terminal_state:
                node_id = leaf_node_id
                number_of_child = len(self.tree[node_id]['child'])
                if not number_of_child:
                    leaf_node_id = node_id
                    is_terminal_state = True
                else:
                    max_value_score = -math.inf
                    best_action = leaf_node_id
                    for i in range(number_of_child):
                        action = self.tree[node_id]['child'][i]
                        child_id = leaf_node_id + (action,)
                        current_value = self.get_value(child_id)
                        if current_value > max_value_score:
                            max_value_score = current_value
                            best_action = action
                    if self.value_type == 'epsilon-greedy' and np.random.sample() < self.epsilon:
                        current_state = self.tree[leaf_node_id]['state']
                        current_board = np.asarray(current_state).reshape(config.rows*config.columns)
                        self.actions_available = [c for c in range(self.config.columns) if not current_board[c]]
                        rand_act = np.random.choice(self.actions_available)
#                         print(rand_act)
                        leaf_node_id = leaf_node_id + (rand_act,)
                    else:
                        leaf_node_id = leaf_node_id + (best_action,)
            return leaf_node_id

        def expansion(self, leaf_node_id):
            '''
            Aim - Add new nodes to the current leaf node by taking a random action
                  and then take a random or follow any policy to take opponent's action.
            '''
            current_state = self.tree[leaf_node_id]['state']
            player_mark = self.tree[leaf_node_id]['player']
            current_board = np.asarray(current_state).reshape(
                config.rows*config.columns)
            self.actions_available = [c for c in range(
                self.config.columns) if not current_board[c]]
            done = check_result(current_state, player_mark, self.config)
            child_node_id = leaf_node_id
            is_availaible = False

            if len(self.actions_available) and not done:
                childs = []
                for action in self.actions_available:
                    child_id = leaf_node_id + (action,)
                    childs.append(action)
                    new_board = put_new_piece(
                        current_state, action, player_mark, self.config)
                    self.tree[child_id] = {'state': new_board, 'player': player_mark,
                                           'child': [], 'parent': leaf_node_id,
                                           'total_node_visits': 0, 'total_node_wins': 0}

                    if check_result(new_board, player_mark, self.config):
                        best_action = action
                        is_availaible = True

                self.tree[leaf_node_id]['child'] = childs

                if is_availaible:
                    child_node_id = best_action
                else:
                    child_node_id = random.choice(childs)

            return leaf_node_id + (child_node_id,)

        def simulation(self, child_node_id):
            '''
            Aim - Reach the final state of the game
            '''
            self.total_parent_node_visits += 1
            state = self.tree[child_node_id]['state']
            previous_player = self.tree[child_node_id]['player']

            is_terminal = check_result(state, previous_player, self.config)
            winning_player = previous_player
            count = 0

            while not is_terminal:

                current_board = np.asarray(state).reshape(
                    config.rows*config.columns)
                self.actions_available = [c for c in range(
                    self.config.columns) if not current_board[c]]

                if not len(self.actions_available) or count == 3:
                    winning_player = None
                    is_terminal = True

                else:
                    count += 1
                    if previous_player == 1:
                        current_player = 2
                    else:
                        current_player = 1

                    for actions in self.actions_available:
                        state = put_new_piece(
                            state, actions, current_player, self.config)
                        result = check_result(
                            state, current_player, self.config)
                        if result:  # A player won the game
                            is_terminal = True
                            winning_player = current_player
                            break

                previous_player = current_player

            return winning_player

        def backpropagation(self, child_node_id, winner):
            '''
            Aim - Update the traversed nodes
            '''
            player = self.tree[(0,)]['player']

            if winner == None:
                reward = 0
            elif winner == player:
                reward = 1
            else:
                reward = -10

            node_id = child_node_id
            self.tree[node_id]['total_node_visits'] += 1
            self.tree[node_id]['total_node_wins'] += reward

        def start_the_game(self):
            '''
            Aim - Complete MCTS iteration with all the process running for some fixed time
            '''
            self.initial_time = time.time()
            is_expanded = False

            while time.time() - self.initial_time < self.time_limit:
                node_id = self.selection()
                if not is_expanded:
                    node_id = self.expansion(node_id)
                    is_expanded = True
                winner = self.simulation(node_id)
                self.backpropagation(node_id, winner)

            current_state_node_id = (0,)
            action_candidates = self.tree[current_state_node_id]['child']
            total_visits = -math.inf
            for action in action_candidates:
                action = current_state_node_id + (action,)
                visit = self.tree[action]['total_node_visits']
                if visit > total_visits:
                    total_visits = visit
                    best_action = action

            return best_action

    my_agent = MCTS(obs, config)

    return my_agent.start_the_game()[1]

### Validate Agent

In [ ]:
env = make("connectx", debug=True)

env.reset()

env.run([connectx_agent, 'negamax'])
x = env.render(mode="ipython", width=500, height=450)

### Задание:
Реализовать UCB стратегию и сравнить результат с жадной и эпсилон-жадной
<img src = "https://miro.medium.com/max/6000/1*WcFwHS3n2nXcWQILZqZ5Gw.png" style="width: 500px;">

<!-- <img src = "https://miro.medium.com/max/1725/1*K5v2P5A7p9f35d_nFcwb9A.png" style="width: 500px;"> -->

## Часть 5: AlphaGo Zero как идея и что в нем особенного

### AlphaGo
В общем и целом, AlphaGo опирается на MCTS. Ключевое отличие — на этапе симуляции при добавлении нового узла, для определения насколько она хорошая, вместо случайных rollout'ов используется нейросеть.

 1. Тренируюся две сети, каждая из которых получает на вход состояние доски и говорит, какой бы ход в этой ситуации сделал человек. Почему две? Потому что одна — медленная, но работает хорошо (57% верных предсказаний, и каждый дополнительный процент даёт очень солидный бонус к итоговому результату), а вторая обладает намного меньшей точностью, зато быстрая. Обе эти сети тренируюся на демонстрация, полученных из партий сильных игроков.

 2. После чего обе сети играют между собой.

 3. Тренируется value-сеть, которая получает на вход текущее состояние доски, а в ответ отдаёт число от -1 до 1 — вероятность выиграть, оказавшись в этой позиции в какой-то момент партии.

В итоге получается что есть одна медленная и точная функция, которая говорит, куда надо ходить , одна быстрая функция, которая делает то же самое, хоть и не так хорошо, и третья функция, которая, глядя на доску, говорит, проиграешь ты или выиграешь, если окажешься в этой ситуации. Для чего это нужно?
Иcпользуем MCTS и используем первую, в качестве стратегии по дереву, вторую — в качестве случайной стратегии, а третью — чтобы напрямую без rollout'а оценить, насколько хорош узел.

### Основные проблемы AlphaGo

 1. Для стартового обучения используются демонстрации. Получается, что без человеческого интеллекта искусственный интеллект не работает.
 2. Много заинженеренных фич. Например, сеть, которая определяет следующий ход, помимо непосредственно текущего состояния получает следующее:
     * сколько ходов назад был поставлен тот или иной камень;
     * сколько свободных точек вокруг данного камня;
     * сколько своих камней ты пожертвуешь, если сходишь в данную точку;
     * легален ли вообще данный ход, то есть позволяется ли он правилами го;
     * поучаствует ли камень, поставленный в эту точку, в так называемом “лестничном” построении;
     * и так далее — в общей сложности 48 слоёв с информацией. А “быстрой” сети, которая предсказывает вероятность победы, и вовсе отдают на вход сто с лишним тысяч заготовленных параметров. Получается, модель учится не играть в го per se, а показывать результаты в некотором заранее очень хорошо подготовленном окружении с большим количеством свойств, о которых ей рассказывает опять же человек.

 3. Нужен здоровый кластер, чтобы всё это запустить.

### AlphaGo Zero
Изменения, которые произошли в новой версии:
 1. Объединили две сети из прошлых версий AlphaGo в одну. Она получает состояние доски с небольшим количеством фич, и в конце два её выхода выдают два результата: policy-выход выдаёт массив 19х19, который показывает, насколько вероятен каждый из ходов из данной позиции, а value выдаёт одно число — вероятность выиграть партию, опять же из данной позиции.

2. Поменятли сам RL-алгоритм. Если раньше непосредственно MCTS использовался только во время игры, то теперь он используется сразу при тренировке.

<img src='https://habrastorage.org/getpro/habr/post_images/1fd/853/b13/1fd853b13eed794443a5f1d693d0a5f7.png' style="width: 500px;">
    
В каждом узле дерева состояний хранится четыре значения — **N** (сколько раз был посещен узел), **V** (value), **Q** (усреднённое value всех дочерних узлов) и **P** (вероятность, что из всех допустимых на данном ходу узлов будет выбран именно этот). Алгоритм:

* Берётся дерево, корнем которого является текущий узел.
* Выбираетсятот дочерний узел, где больше значение **Q + U** (**U** — параметр, стимулирующий исследовани).
* После достижения конца дерева — состояния, когда дочерних узлов нет, а игра ещё не закончена, это состояние передается на вход нейросети, в ответ получает **v** (value текущей ноды) и **p** (вероятности следующих ходов).
* **v** записывается в узел.
* Создаются дочерние узлы с **P** согласно **p** и нулевыми **N**, **V** и **Q**.
* Обновляются все узлы выше текущего, которые были выбраны во время симуляции, следующим образом: **N := N + 1**; **V := V + v**; **Q := V / N**.


А дальше ход, который сеть действительно сделает, выбирается одним из двух способов:

— Если это реальная игра, то выбираем узела, где больше **N**;
— Если просто тренировка, выбираем ход из распределения **Pi ~ N ^ (1/T)**, где **T** — просто некоторая температура для контроля баланса между исследованием и эффективностью.

<img src='https://habrastorage.org/getpro/habr/post_images/abd/13c/3a8/abd13c3a88e0b76740ca83b5fb21a982.png' style="width: 500px;">
Если подробнее, то предположим что имеется некоторая «наилучшая» сеть с весами А. Эта сеть A играет сама с собой 25 000 раз (используя MCTS со своими весами для оценки новых узлов), и для каждого хода сохраняется состояние, распределение $\Pi$ и то, чем закончилась игра (+1 за победу и -1 за поражение). Дальше батчи из 2048 случайных позиций из последних 500 000 игр, отдаём 1000 таких батчей на тренировку и получаем некоторую новую сеть с весами B, после чего сеть A играет 400 игр с сетью B — при этом обе сети используют MCTS для выбора хода, только при оценке новой ноды A, очевидно, использует свои веса, а B — свои. Если B побеждает более, чем в 55% случаев, она становится лучшей сетью, если нет — чемпион остаётся прежним.

## Часть 6: Глубокое планирование как новое направление

Важными недостатками MCTS подхода являются:
 1. Сложность распараллеливания;
 2. Отсутствие end-to-end обучения, что значительно сказывается на скорости обучения;
 3. Тяжело работать с непрерывными действиями(как и с большим пространством действий);
 4. Слабые результаты при большом пространстве состояний.

### PlaNet

В 2018 году Google AI в сотрудничестве с DeepMind представили алгоритм Deep Planning Network (PlaNet), который изучает модель мира на основе входных изображений и успешно использует ее для планирования. PlaNet решает множество задач управления на основе изображений, конкурируя с продвинутыми агентами без моделей с точки зрения конечной производительности, при этом обеспечивая большую эффективность обработки данных.

PlaNet изучает динамическую модель на основе входных данных изображения и эффективно планирует с ее помощью для получения нового опыта. Основной идеей является использование скрытых состояний. Вместо планирования по изображениям используется скрытый слой, таким образом, агент может автоматически изучать более абстрактные представления, такие как положения и скорости объектов, что упрощает планирование.
<img src='https://1.bp.blogspot.com/-MDnZ7uyo-VQ/XGX13O1eahI/AAAAAAAADwg/_ObesnP5S_8Tx5nwrzdigKlIX2raW8OewCLcBGAs/s1600/image1.png' style="width: 500px;">

Кодирование и декодирование изображений (трапеции на рисунке выше) требует значительных вычислений, что замедляет планирование. Однако в пространстве скрытых состояний планирование происходит значительно быстрее, ввиду того, что для оценки последовательности действий необходимо предсказывать только награды. 

<img src='https://3.bp.blogspot.com/-XM_o5v3AOB4/XGX2VKrI8HI/AAAAAAAADwo/preHUCFZ1IAp2qmtaSabre3Pzo9wcr7GACLcBGAs/s1600/image5.png' style="width: 500px;">

Код и примеры для запуска:
https://github.com/google-research/planet

### Dreamer

Планирование предложенное в PlaNet требовательно к вычислениям и не полностью использует модель динамики среды. 
Более того, даже глубокие модели мира ограничены в том, насколько далеко вперед они могут точно предсказать. Dreamer преодолевает эти ограничения, используя value-network и actor-network.

Dreamer учит actor-network предсказывать успешные действия. Благодаря этому Dreamer понимает как небольшие изменения в действиях влияют на то, какие награды предсказываются в будущем, позволяя ему усовершенствовать actor-network в направлении, которое больше всего увеличивает награды. Чтобы учесть награды за пределами горизонта прогнозирования, value-network оценивает сумму будущих вознаграждений для каждого состояния модели.

<img src='https://1.bp.blogspot.com/-wI4n0f2_2ZQ/XnFFcl9OjqI/AAAAAAAAFgE/dxkupxZvxr0swvc4Qlc6BBuXgGpLdF6PQCEwYBhgL/s1600/image5.gif' style="width: 500px;">

<!-- Dreamer учится дальновидному поведению на основе предсказанных последовательностей состояний модели. Сначала он изучает долгосрочное значение (v̂2 – v̂3) каждого состояния, а затем прогнозирует действия (â1 – â2), которые приводят к высоким вознаграждениям и значениям, путем обратного распространения их через последовательность состояний в actor-network. -->

Dreamer отличается от PlaNet, не смотря на идентичную модель среды.  PlaNet ищет лучшее действие среди множества прогнозов для различных последовательностей действий. Dreamer же, напротив, не использует этот дорогостоящий поиск, разделяя планирование и действие. После того, как actor-network обучена, она вычисляет действия для взаимодействия со средой без дополнительного поиска. Кроме того, Dreamer рассматривает вознаграждения за пределами горизонта планирования с помощью value-network и использует обратное распространение для эффективного планирования.

Код и примеры для запуска:
https://github.com/google-research/dreamer

## Источники
1. https://pranav-agarwal-2109.medium.com/game-ai-learning-to-play-connect-4-using-monte-carlo-tree-search-f083d7da451e
2. https://habr.com/ru/post/343590/
3. https://ai.googleblog.com/2019/02/introducing-planet-deep-planning.html
4. https://ai.googleblog.com/2020/03/introducing-dreamer-scalable.html
5. https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf
6. https://link.springer.com/content/pdf/10.1007/11871842_29.pdf
7. https://www.kaggle.com/c/connectx
8. https://arxiv.org/abs/2012.04603